In [1]:
import pandas as pd
import pickle
import numpy as np
#from tqdm import tqdm

In [9]:
valor_utm = {2023: 61769,
             2022: 54442,
             2021: 50978,
             2020: 49673,
             2019: 48353,
             2018: 47019,
             2017: 46229,
             2016: 44955,
             2015: 43198,
             2014: 40935
            }
mapper = {"Rol_size":"Predios H", 
         "aseo_nunique":"Cuotas",
         "aseo_max":"Maximo",
          "aseo_min":"Minimo",
          "aseo_mean":"Cuota promedio",
         "Exencion_DA_sum":"Exentos"}

In [17]:
diccionario=pd.read_excel("../Datos/09_dictionaries/codigos_comunas.xls")
#diccionario

In [18]:
with open("../Datos/09_dictionaries/glosario_columnas_sinim.pkl", "rb") as file:
    glosario= pickle.load(file)
glosario

{'BPIGM': ' (M$) Presupuesto Inicial Gastos Municipales',
 'BPISICMUN': ' (M$) Presupuesto Inicial Saldo Inicial de Caja sector Municipal',
 'BPIIM': ' (M$) Presupuesto Inicial Sector Municipal',
 'BPVGM': ' (M$) Presupuesto Vigente Gastos Municpiales',
 'BPVMSALINI': ' (M$) Presupuesto Vigente Saldo Inicial de Caja Sector Municipal',
 'BPVIM': ' (M$) Presupuesto Vigente Sector Municipal',
 'IADM83': ' (M$) Casinos de Juegos Ley Nº19.995.',
 'IADM97': ' (M$) Derechos de Aseo ',
 'IADM98': ' (M$) Derechos de Aseo Cobro Directo y de Patentes Comerciales',
 'IADM99': ' (M$) Derechos de Aseo por Impuesto Territorial',
 'IADM10': ' (TAS) Disponibilidad Presupuestaria Municipal por Habitante (M$)',
 'IADM140': ' (M$) Impuesto Territorial de Beneficio Municipal (Art. 37 DL 3063) ',
 'IADM01': ' (M$) Ingresos Municipales (Ingreso Total Percibido)',
 'IADM999': ' (M$) Ingresos Municipales (Ingreso Total Percibido) sin Saldo Inicial de Caja',
 'IADM40': ' (M$) Ingresos por Fondo Común Municipal'

In [39]:
path= "../Datos/03_unidos"
df_sinim=pd.read_csv(path+"/sinim_2014_2019_consolidado.csv")
df=pd.DataFrame()
for año in [2014,2015,2016,2017,2018,2019]:
    df_sii= pd.read_csv(path+f"/{año}.csv")
    df_sii= df_sii.merge(diccionario[["CONARASII","Nombre Comuna","Region"]], how="left",left_on="Comuna",right_on="CONARASII")
    logic = (df_sii["Semestre"]==1) & (df_sii["Destino"]=="H")
    utm=valor_utm[año]
    vex_DA= 225*utm
    df_sii["Exencion_DA"]= np.where(df_sii["Avaluo_Fiscal"]<=vex_DA,1,0)
    df_sii["aseo"]=np.where(df_sii["Aseo"]>0,df_sii["Aseo"]*2,np.nan)
    df_DA=df_sii.loc[logic].groupby(["Año","Region","Nombre Comuna"]).agg({"Rol":"size","aseo":["nunique","max","min","mean"], "Exencion_DA":"sum"})
    df_DA.columns=["_".join(col) for col in df_DA.columns.values]
    df_DA.rename(mapper=mapper, axis=1, inplace=True)
    df_DA=df_DA.reset_index(drop=False)
    df=pd.concat([df,df_DA],axis=0)

In [40]:
revision=df.merge(df_sinim[["municipio","año","ICAR004","IADM98","BSASC","IADM41","IADM42"]],how="left",left_on=["Año","Nombre Comuna"], right_on=["año","municipio"])
Datos_DA=revision[["Año","Region","Nombre Comuna","Predios H","Cuotas","Maximo","Minimo","Cuota promedio","Exentos","ICAR004","IADM98","BSASC","IADM41","IADM42"]]

In [41]:
Datos_DA.rename(mapper={"ICAR004":"Poblacion","IADM98":"Cobro directo","BSASC":"Gasto SINIM","Nombre Comuna":"Comuna","IADM41":"IPP","IADM42":"IP"}, axis=1, inplace=True)
Datos_DA["Gasto SINIM"]=Datos_DA["Gasto SINIM"]*1000
Datos_DA["IPP"]=Datos_DA["IPP"]*1000
Datos_DA["IP"]=Datos_DA["IP"]*1000
Datos_DA["Cobro directo"]=Datos_DA["Cobro directo"]*1000
Datos_DA["Gasto Aseo"]=(100*Datos_DA["Cuota promedio"]*Datos_DA["Predios H"])
Datos_DA["Predios CD"]=((Datos_DA["Cobro directo"])/Datos_DA["Cuota promedio"]).round(1)
Datos_DA["% Exentos"]=(100*Datos_DA["Exentos"]/Datos_DA["Predios H"]).round(1)
Datos_DA["Costo Exencion"]=(Datos_DA["Exentos"]*Datos_DA["Minimo"])
Datos_DA["% en IP"]=(100*Datos_DA["Costo Exencion"]/Datos_DA["IP"]).round(1)
Datos_DA.to_excel("../Datos/04_datos/Datos_DA.xlsx")
Datos_DA

/var/folders/_7/b1d_qnnd5gs3d5fd0wbclzjw0000gn/T/ipykernel_21430/3000356323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_DA.rename(mapper={"ICAR004":"Poblacion","IADM98":"Cobro directo","BSASC":"Gasto SINIM","Nombre Comuna":"Comuna","IADM41":"IPP","IADM42":"IP"}, axis=1, inplace=True)
/var/folders/_7/b1d_qnnd5gs3d5fd0wbclzjw0000gn/T/ipykernel_21430/3000356323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_DA["Gasto SINIM"]=Datos_DA["Gasto SINIM"]*1000
/var/folders/_7/b1d_qnnd5gs3d5fd0wbclzjw0000gn/T/ipykernel_21430/3000356323.py:3: Setting

,Año,Region,Comuna,Predios H,Cuotas,Maximo,Minimo,Cuota promedio,Exentos,Poblacion,Cobro directo,Gasto SINIM,IPP,IP,Gasto Aseo,Predios CD,% Exentos,Costo Exencion,% en IP
0,2014,ANTOFAGASTA,ANTOFAGASTA,89996,1,45644.0,45644.0,45644.0,28039,372325,415750000.0,5.665204e+09,3.620422e+10,4.218774e+10,4.107777e+11,9108.5,31.2,1.279812e+09,3.0
1,2014,ANTOFAGASTA,CALAMA,39114,1,44476.0,44476.0,44476.0,10852,173691,835178000.0,3.601925e+09,1.387053e+10,1.759828e+10,1.739634e+11,18778.2,27.7,4.826536e+08,2.7
2,2014,ANTOFAGASTA,MARÍA ELENA,89,0,NaN,NaN,NaN,86,5340,0.0,1.309000e+06,1.953576e+09,2.465453e+09,NaN,NaN,96.6,NaN,NaN
3,2014,ANTOFAGASTA,MEJILLONES,2651,1,40000.0,40000.0,40000.0,2276,11090,36599000.0,3.960360e+08,4.432319e+09,5.347886e+09,1.060400e+10,915.0,85.9,9.104000e+07,1.7
4,2014,ANTOFAGASTA,OLLAGÜE,83,0,NaN,NaN,NaN,74,315,0.0,1.506960e+08,1.018720e+08,1.150210e+09,NaN,NaN,89.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2065,2019,ÑUBLE,SAN IGNACIO,2935,1,19176.0,19176.0,19176.0,2394,16598,7256000.0,3.799900e+07,3.879030e+08,3.341810e+09,5.628156e+09,378.4,81.6,4.590734e+07,1.4
2066,2019,ÑUBLE,SAN NICOLÁS,1680,0,NaN,NaN,NaN,1292,12068,2777000.0,9.547600e+07,8.721380e+08,2.815512e+09,NaN,NaN,76.9,NaN,NaN
2067,2019,ÑUBLE,TREHUACO,936,0,NaN,NaN,NaN,768,5667,0.0,1.500100e+08,2.285000e+08,2.040034e+09,NaN,NaN,82.1,NaN,NaN
2068,2019,ÑUBLE,YUNGAY,4142,0,NaN,NaN,NaN,2887,18499,24167000.0,1.420730e+08,8.337690e+08,4.248037e+09,NaN,NaN,69.7,NaN,NaN
